In [1]:
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.data import InMemoryDataset
import pickle

In [2]:
import torchgeometry as tgm

In [3]:
focal_loss = tgm.losses.FocalLoss(alpha=0.5, gamma=2.0, reduction='mean')

In [4]:
no_classes=2
inputs = torch.randn(1, no_classes, 1, 2, requires_grad=True)
target = torch.empty(1, 1, 2, dtype=torch.long).random_(no_classes)
output = focal_loss(inputs, target)
output.backward()
print(output)

tensor(0.0275, grad_fn=<MeanBackward0>)


In [5]:
 
with open("graphs_data", "rb") as fp:   # Unpickling
    read_graph_data = pickle.load(fp)

In [6]:
data_list = []
count = 0
for data in read_graph_data:
    
    if data.x.shape[0] == 0:
        continue
        
    data_list.append(data)

        
    

In [7]:
len(data_list)

628

In [8]:
train_loader = DataLoader(data_list, batch_size=16,shuffle=True)
test_loader = DataLoader(data_list, batch_size=16,shuffle=False)

In [9]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
#         torch.manual_seed(12345)
#         self.conv1 = CGConv((mydataset.num_node_features, hidden_channels),mydataset.num_edge_features,batch_norm=True)
        self.conv1 = GCNConv(1, hidden_channels)
        
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.conv4 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, 2)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)
        x = x.relu()
        x = self.conv4(x, edge_index)
 

        # 2. Readout layer
#         print(x.shape)
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
#         print(x.shape)
        # 3. Apply a final classifier
#         x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x

model = GCN(hidden_channels=64)
print(model)

GCN(
  (conv1): GCNConv(1, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (conv4): GCNConv(64, 64)
  (lin): Linear(in_features=64, out_features=2, bias=True)
)


In [10]:
model = GCN(hidden_channels=64).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
# weights = torch.tensor([.05,.95]).to(device)
focal_loss = tgm.losses.FocalLoss(alpha=0.5, gamma=2.0, reduction='mean')

In [11]:
def train():
    model.train()
    for batch_idx,data in enumerate(train_loader):



        data = data.to(device)
        out = model(data.x, data.edge_index,data.batch)
        
#         print(out.reshape(-1,2,1,1).shape)
#         print(data.y.reshape(-1,1,1).shape)
#         break

        loss = focal_loss(out.reshape(-1,2,1,1), data.y.reshape(-1,1,1))  # Compute the loss.
#         break
        loss.backward()  # Derive gradients.
        
        optimizer.step()
        optimizer.zero_grad()
#             print(batch_idx)
        

    return loss.cpu().item()
    
def test(loader):
    model.eval()

    correct = 0
    for data in loader:  # Iterate in batches over the training/test dataset.
        


        data.edge_attr = data.edge_attr.float() 
        data = data.to(device)
        out = model(data.x, data.edge_index, data.batch)  
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        

        correct += int((pred == data.y).sum())  # Check against ground-truth labels.
    return correct / len(loader.dataset)  # Derive ratio of correct predictions.

test(test_loader)
# train()

0.45222929936305734

In [12]:

for epoch in range(1, 400):
    loss = train()
#     break
    train_acc = test(test_loader)
#     test_acc = test(test_loader)
    if epoch % 5 == 0: 
        print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, loss : {loss:.4f}')

Epoch: 005, Train Acc: 0.7229, loss : 0.0717
Epoch: 010, Train Acc: 0.7229, loss : 0.0422
Epoch: 015, Train Acc: 0.7611, loss : 0.0443
Epoch: 020, Train Acc: 0.7771, loss : 0.0368
Epoch: 025, Train Acc: 0.8217, loss : 0.0266
Epoch: 030, Train Acc: 0.8105, loss : 0.0424
Epoch: 035, Train Acc: 0.8137, loss : 0.0551
Epoch: 040, Train Acc: 0.8153, loss : 0.0541
Epoch: 045, Train Acc: 0.8535, loss : 0.0499
Epoch: 050, Train Acc: 0.8424, loss : 0.0268
Epoch: 055, Train Acc: 0.8392, loss : 0.0086
Epoch: 060, Train Acc: 0.8487, loss : 0.0514
Epoch: 065, Train Acc: 0.8615, loss : 0.0176
Epoch: 070, Train Acc: 0.8455, loss : 0.0148
Epoch: 075, Train Acc: 0.8487, loss : 0.0345
Epoch: 080, Train Acc: 0.8551, loss : 0.0727
Epoch: 085, Train Acc: 0.8344, loss : 0.0506
Epoch: 090, Train Acc: 0.8439, loss : 0.0192
Epoch: 095, Train Acc: 0.8694, loss : 0.0635
Epoch: 100, Train Acc: 0.8631, loss : 0.0487
Epoch: 105, Train Acc: 0.8854, loss : 0.0240
Epoch: 110, Train Acc: 0.8025, loss : 0.0425
Epoch: 115

In [9]:
import numpy as np
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([p.numel() for p in model_parameters])
params

12738